<a href="https://colab.research.google.com/github/Ricardo0621/Drosophila/blob/master/maDrosophila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590444465547-SHXODUII311HEE407IL6/ke17ZwdGBToddI8pDm48kE4VnnB9_j2k1VP236ADqAFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxQg9Vf0owGyf3dhfDKy8SxMujaKmp2B54Sb3VS1rO76Whq-cUhHVuKFlGUXsU9tJk/ezgif.com-video-to-gif.gif?format=1500w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- cross evaluate inference parameters
- analyze novel videos
- assemble tracklets
- create quality check plots

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub.

Mathis et al, in prep. <- please note, we are providing this toolbox as an early access release; more feeatures and details will be released with the forthcoming paper.




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
#(this will take a few minutes to install all the dependences!)
!pip install git+https://github.com/DeepLabCut/DeepLabCut.git
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

  Cloning https://github.com/DeepLabCut/DeepLabCut.git to /tmp/pip-req-build-alh5137b
  Running command git clone -q https://github.com/DeepLabCut/DeepLabCut.git /tmp/pip-req-build-alh5137b
  Created wheel for deeplabcut: filename=deeplabcut-2.2b8-cp36-none-any.whl size=668047 sha256=7a14524912023fca97386a06e1c7ceebc247efe3cabc068861b7edb0aa3298c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-yf0cre4o/wheels/57/9f/a0/9ed45695cd4d222c82264d0c3cc2d0bcb0dec399fbfbd49264
Successfully built deeplabcut


In [ ]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#GUIs don't work on the cloud, so we supress them:
import os
os.environ["DLClight"]="True"
import deeplabcut

# Suppress tensorflow warning messages:
import tensorflow as tf

vers = (tf.__version__).split(".")
if int(vers[0]) == 1 and int(vers[1]) > 12:
    TF = tf.compat.v1  # behaves differently before 1.13
else:
    TF = tf

TF.logging.set_verbosity(TF.logging.ERROR)
DEBUG = True and "DEBUG" in os.environ and os.environ["DEBUG"]
from deeplabcut import DEBUG

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data):

### First, place your porject folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [ ]:
# PLEASE EDIT THIS:
  
ProjectFolderName = 'maDrosophila'
VideoType = 'mov' #, mp4, MOV, or avi, whatever you uploaded!


# we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/drive/My Drive/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path

#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/content/drive/My Drive/'+ProjectFolderName+'/config.yaml'
path_config_file

'/content/drive/My Drive/maDrosophila/config.yaml'

## Create a multi-animal training dataset:
### You must do this step inside of Colab:

- Reminder: you must connect EVERY bodypart in a skeleton before you run this step! See docs for crucial details on how to do this effciently: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#b-configure-the-project-

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589256735280-SCN7CROSJNJWCDS6EK5T/ke17ZwdGBToddI8pDm48kB08p9-rNkpPD7A3fw8YFjZZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZamWLI2zvYWH8K3-s_4yszcp2ryTI0HqTOaaUohrI8PIno0kSvzOWihTW1zp8-1-7mzYxUQjsVr2n3nmNdVcso4/bodyparts-skeleton.png?format=1000w)
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1589410182515-9SJO9MML6CNCXBAWQ6Z6/ke17ZwdGBToddI8pDm48kJ1oJoOIxBAgRD2ClXVCmKFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxBw7VlGKDQO2xTcc51Yv6DahHgScLwHgvMZoEtbzk_9vMJY_JknNFgVzVQ2g0FD_s/ezgif.com-video-to-gif+%2811%29.gif?format=750w)

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
deeplabcut.cropimagesandlabels(path_config_file, size=(400, 400), userfeedback=False)

#if you labeled on Windows, please set the windows2linux=True:
deeplabcut.create_multianimaltraining_dataset(path_config_file, windows2linux=False)

/content/drive/My Drive/maDrosophila/training-datasets/iteration-0/UnaugmentedDataSet_maDrosophilaoct1  already exists!
Utilizing the following graph: [[1, 2], [0, 1], [1, 3], [1, 4], [2, 3], [0, 4], [0, 3], [3, 4], [0, 2]]


  0%|          | 0/330 [00:00<?, ?it/s]

Creating training data for  1 0.95
This can take some time...


100%|██████████| 330/330 [01:50<00:00,  2.98it/s]


The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. 
#Typically, you want to train to 50,000 iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

#which shuffle do you want to train?
shuffle = 1
deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100,saveiters=1000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['leftwing', 'rightwing', 'head', 'torax', 'abdomen'],
 'batch_size': 8,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_maDrosophilaoct1/maDrosophila_RicardoDiaz95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'display_iters': 500,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_maDrosophilaoct1/Documentation_data-maDrosophila_95shuffle1.pickle',
 'min_input_size': 64,
 

Selecting multi-animal trainer
Activating limb prediction...
Starting with multi-animal imaug + adam pose-dataset loader.
Batch Size is 8
Getting specs multi-animal-imgaug 9 5
Initializing ResNet
Loading ImageNet-pretrained resnet_50


KeyboardInterrupt: ignored

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: for maDLC, this is several steps. 
 - First, we evaluate the pose estimation performance, and then we can cross-valudate optimal inference parameters.

- This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [ ]:
#%matplotlib notebook
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True,c_engine=False)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])


#and begin to cross-validate:
deeplabcut.evaluate_multianimal_crossvalidate(
            path_config_file,
            Shuffles=[shuffle],
            edgewisecondition=True,
            leastbpts=1,
            init_points=20,
            n_iter=50,
            target='rpck_train',
        )

Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['leftwing', 'rightwing', 'head', 'torax', 'abdomen'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_maDrosophilaoct1/maDrosophila_RicardoDiaz95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_joints': 5,
 'num_limbs': 9,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_graph'

Running  DLC_resnet50_maDrosophilaoct1shuffle1_20000  with # of trainingiterations: 20000
Initializing ResNet
Activating extracting of PAFs


0it [00:00, ?it/s]

Analyzing data...


/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:280: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
1it [00:01,  1.21s/it]/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:280: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
3it [00:01,  1.14it/s]/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:280: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
6it [00:01,  2.09it/s]/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:280: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:280: RuntimeWarning: All-NaN slice encountered
  distnorm[imagein

...

/content/drive/My Drive/maDrosophila/evaluation-results/iteration-0/maDrosophilaoct1-trainset95shuffle1  already exists!
/content/drive/My Drive/maDrosophila/evaluation-results/iteration-0/maDrosophilaoct1-trainset95shuffle1/DLC_resnet50_maDrosophilaoct1shuffle1_20000-snapshot-20000.h5


Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['leftwing', 'rightwing', 'head', 'torax', 'abdomen'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_maDrosophilaoct1/maDrosophila_RicardoDiaz95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_joints': 5,
 'num_limbs': 9,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_graph'

Computing distances...
rpck nan rpck train: nan
rmse nan miss nan hit nan
rpck 0.1513435713636014 rpck train: 0.7215027519285384
rmse 9.977305917991135 miss 0.0 hit 3.0
rpck nan rpck train: nan
rmse nan miss nan hit nan
rpck 0.030743161061692684 rpck train: 0.4200585974700132
rmse 10.70729920176197 miss 1.5 hit 2.0
rpck 0.1513435713636014 rpck train: 0.7199870532085919
rmse 9.977305917991135 miss 0.0 hit 3.0
rpck 0.11706751040393416 rpck train: 0.650552987329071
rmse 10.19044381427476 miss 0.2222222222222222 hit 3.3333333333333335
rpck 0.08644067458816679 rpck train: 0.6165709599598467
rmse 10.36195904740147 miss 0.375 hit 3.375
rpck 0.1364180362127464 rpck train: 0.6678166592830114
rmse 9.98753845468232 miss 0.0 hit 3.5555555555555554
rpck nan rpck train: nan
rmse nan miss nan hit nan
rpck 0.19441820480086214 rpck train: 0.6328860785984017
rmse 9.19390174801638 miss 0.0 hit 4.5
rpck 0.1513435713636014 rpck train: 0.7183589566187789
rmse 9.977305917991135 miss 0.0 hit 3.0
rpck nan rpck

Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['leftwing', 'rightwing', 'head', 'torax', 'abdomen'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_maDrosophilaoct1/maDrosophila_RicardoDiaz95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_joints': 5,
 'num_limbs': 9,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_graph'

rpck 0.1364180362127464 rpck train: 0.6678166592830114
rmse 9.98753845468232 miss 0.0 hit 3.5555555555555554
/content/drive/My Drive/maDrosophila/evaluation-results/iteration-0/maDrosophilaoct1-trainset95shuffle1/DLC_resnet50_maDrosophilaoct1shuffle1_20000-snapshot-20000.h5
Saving optimal inference parameters...
   train_iter  train_frac  shuffle  rmse_train  hits_train  misses_train  falsepos_train  ndetects_train  pck_train  rpck_train  rmse_test  hits_test  misses_test  falsepos_test  ndetects_test  pck_test  rpck_test
0     20000.0        95.0      1.0     2.41524    4.140097      0.091787        0.280193        1.294686   0.950941    0.865091   9.977306        3.0          0.0       0.214286            1.0  0.188095   0.313843


^ NOTE: this optimized part detection for video analysis. It cannot optimze for tracking, as this is use-case dependent. Please check the docs on how you can set the best parameters and modify/test before "final" tracking parameters. You can use COLAB to analyze videos, but afterwards we recommend using the outputs/proejct folder locally to run the final steps! They do not require a GPU. 

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
print("Start Analyzing my video(s)!")
scorername = deeplabcut.analyze_videos(path_config_file, 
                                       videofile_path, 
                                       shuffle=shuffle, 
                                       videotype=VideoType, 
                                       c_engine=False)

Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['leftwing', 'rightwing', 'head', 'torax', 'abdomen'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_maDrosophilaoct1/maDrosophila_RicardoDiaz95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_joints': 5,
 'num_limbs': 9,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_graph'

Start Analyzing my video(s)!
Using snapshot-20000 for model /content/drive/My Drive/maDrosophila/dlc-models/iteration-0/maDrosophilaoct1-trainset95shuffle1
Initializing ResNet
Activating extracting of PAFs
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/maDrosophila/videos/Movie.S3.mov
/content/drive/My Drive/maDrosophila/videos  already exists!
Loading  /content/drive/My Drive/maDrosophila/videos/Movie.S3.mov


  0%|          | 0/301 [00:00<?, ?it/s]

Duration of video [s]:  10.04 , recorded with  29.97 fps!
Overall # of frames:  301  found with (before cropping) frame dimensions:  1280 720
Starting to extract posture


310it [00:30, 11.86it/s]                         

Detected frames:  301


310it [00:30, 10.00it/s]

Saving results in /content/drive/My Drive/maDrosophila/videos...
The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


## The steps below work on a single video at a time.
- Here you can create a video to check the pose estimation detection quality! If this looks good, proceed to tracklet conversions with the interactive GUI (ouside of COLAB for now), or if you know your optimal parameters, you can automate this and run the additional steps shown in a few cells down.

In [ ]:
# PROTIP: look at the output video; if the pose estimation (i.e. key points) don't look good, don't proceed with tracking - add more data to your training set and re-train!
from deeplabcut.utils import auxiliaryfunctions
#scorername, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(path_config_file, shuffle, trainFraction=0)
print("scorename is: "+scorername)

#inferencefg_path = '/content/drive/My Drive/mwm-penguins-2020-03-31/dlc-models/iteration-0/mwmMar31-trainset95shuffle1/test/inference_cfg.yaml'
#scorername, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(
 #   inferencefg_path, shuffle, trainFraction=0
#)

#let's check a specific video:
Specific_videofile = '/content/drive/My Drive/maDrosophila/videos/Movie.S3.mov'

deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], scorername)

  6%|▌         | 17/301 [00:00<00:01, 163.10it/s]

scorename is: DLC_resnet50_maDrosophilaoct1shuffle1_20000
Creating labeled video for  Movie.S3


100%|██████████| 301/301 [00:03<00:00, 83.42it/s]


## Convert Detections to Tracklets:

- The idea is that you test and adapt hyperparameters for tracking outside of COLAB. Once you have good parameters, this can be automated on future videos. Shown here!

- I.e., instead of always doing an interactive parameter setting step, you can simply convert tracklets to .h5 files using these parameters (see GitHub for more info).

In [ ]:
#assemble tracklets:
#read the docs: which tracker to test out (you can run this twice to try multiple):
tracktype= 'box' #box, skeleton

deeplabcut.convert_detections2tracklets(path_config_file, Specific_videofile, videotype=VideoType,
                                                    shuffle=shuffle, track_method=tracktype, overwrite=True)

## Now you should manually verify the tracks and correct them if needed! [currently only working outside of COLAB]

In [ ]:
''' here is the code you would need:
os.environ["DLClight"]="False"
import deeplabcut

#ATTENTION:
picklefile = '/...._10000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/yourVIDEO.mp4'
#if you want occlusions filled in, tell us how many frames to fill in, i.e. if there is a gap in data:
framestofill = 10. #note, put "0" if you want ALL gaps filled!

%matplotlib inline

from deeplabcut.refine_training_dataset.tracklets import refine_tracklets
TrackletManager, TrackletVisualizer = refine_tracklets(path_config_file, 
                                                          picklefile, 
                                                          Specific_videofile, 
                                                          min_swap_frac=0,
                                                          min_tracklet_frac=0, 
                                                          max_gap=framestofill)
'''

## Let's assume you have great tracking parameters, and you want to analyze a full set of videos:

In [ ]:
#^^^^^^^^^You do NOT neeed to run if you hit "save" in the GUI ^^^^^^^^^^
#this is just if you want to run the same parameters over a set of videos!
picklefile = '/content/drive/My Drive/mwm-penguins-2020-03-31/videos/penguindemoDLC_resnet50_mwmMar31shuffle1_22000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/content/drive/My Drive/mwm-penguins-2020-03-31/videos/penguindemo.mp4'

deeplabcut.convert_raw_tracks_to_h5(path_config_file, picklefile)
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType, 
                                 track_method = tracktype)

## Create plots of your trajectories:

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, track_method=tracktype)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                draw_skeleton=True, 
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True, 
                                track_method = tracktype)